<a href="https://colab.research.google.com/github/Etiene-vc/Quantum-computing/blob/main/Bernstein_Vazirani_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação do Qiskit.


In [ ]:
!pip install qiskit

In [15]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

def oracle(s):
    """
    Cria o circuito do oráculo para a string secreta 's'.

    Args:
        s (str): A string secreta binária (ex: '10110').

    Returns:
        QuantumCircuit: O circuito do oráculo.
    """
    n = len(s)
    qc = QuantumCircuit(n+1) # n qubits para a entrada + 1 qubit auxiliar

    # Itera sobre cada bit da string secreta
    for i in range(n):
        if s[i] == '1':
            qc.cx(i, n) # Aplica uma porta CNOT controlada pelo i-ésimo qubit, afetando o qubit auxiliar

    return qc

def bernstein_vazirani(n, s):
    """
    Implementa o algoritmo de Bernstein-Vazirani.

    Args:
        n (int): O número de qubits.
        s (str): A string secreta binária.

    Returns:
        str: A string secreta medida.
    """
    qc = QuantumCircuit(n+1, n) # n qubits para entrada, 1 qubit auxiliar, n bits clássicos para medição

    # Inicialização: Superposição nos qubits de entrada e |1⟩ no qubit auxiliar
    qc.h(range(n)) # Aplica portas Hadamard a todos os qubits de entrada
    qc.x(n)       # Coloca o qubit auxiliar no estado |1⟩
    qc.h(n)       # Aplica porta Hadamard ao qubit auxiliar

    # Oráculo: Aplica o circuito do oráculo
    qc.append(oracle(s), range(n+1))

    # Transformada de Hadamard: Aplica portas Hadamard novamente nos qubits de entrada
    qc.h(range(n))

    # Medição: Mede os qubits de entrada
    qc.measure(range(n), range(n))


    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator, optimization_level=3) # otimização de circuito
    job = simulator.run(compiled_circuit, shots=1) # Executa com 1 shot (suficiente para o algoritmo)
    result = job.result()
    counts = result.get_counts(0) # Obtém os resultados da medição
    measured_s = list(counts.keys())[0] # Extrai a string medida da chave do dicionário

    return measured_s


# Exemplo de uso
n = 5
s = '10110'
measured_s = bernstein_vazirani(n, s)
print(f"String secreta: {s}")
print(f"String medida: {measured_s}")

if measured_s == s:
    print("O algoritmo funcionou corretamente!")
else:
    print("O algoritmo falhou!")

String secreta: 10110
String medida: 01101
O algoritmo falhou!
